In [1]:
# 실시간 이미지 캡처 및 처리를 이해 opencv 이용

In [2]:
import ultralytics

ultralytics.checks() # 실행환경 체크

Ultralytics YOLOv8.2.52  Python-3.9.7 torch-1.13.1+cu116 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16375MiB)
Setup complete  (16 CPUs, 31.9 GB RAM, 483.1/930.8 GB disk)


In [3]:
# 이미지 읽어서 살펴보기
import cv2

# cv2.imread(file_name. flag) : 이미지를 읽어 Numpy 객체로 만듬
# IMREAD_COLOR : color, 투명한 부분 무시
# IMREAD_GRAYSCALE : grayscale
# IMREAD_UNCHANGED : color, 투명한 부분도 읽기

# Numpy 객체로 반환(행, 열, 색상(BGR))

cv2.imread('test.jpg') # COLOR 기본 값

array([[[183, 171, 171],
        [199, 187, 187],
        [204, 192, 192],
        ...,
        [206, 180, 164],
        [202, 176, 160],
        [206, 180, 164]],

       [[187, 175, 175],
        [193, 181, 181],
        [197, 185, 185],
        ...,
        [206, 180, 164],
        [205, 179, 163],
        [211, 185, 169]],

       [[199, 187, 187],
        [197, 185, 185],
        [199, 187, 187],
        ...,
        [211, 185, 169],
        [209, 183, 167],
        [211, 185, 169]],

       ...,

       [[  0,   6,  10],
        [  0,   8,  12],
        [  5,  17,  19],
        ...,
        [  4,  18,  24],
        [  4,  18,  24],
        [ 14,  28,  34]],

       [[  0,  11,  13],
        [  0,  11,  13],
        [  6,  15,  18],
        ...,
        [  9,  23,  29],
        [  9,  23,  29],
        [ 17,  31,  37]],

       [[  3,  15,  17],
        [  0,  10,  12],
        [  0,   8,  11],
        ...,
        [ 17,  31,  37],
        [ 12,  26,  32],
        [ 14,  28,  34]]

In [4]:
# cv2.imshow(title, image) : 특정한 이미지 화면에 출력
# title : 윈도우 창의 제목
# image : 출력할 이미지 객체


# 이미지 불러오기
img = cv2.imread('test.jpg')

# 이미지 크기 조정
resized_img = cv2.resize(img, (500, 500))  # (500, 500)은 원하는 크기

# 이미지 표시하기
cv2.imshow('image', resized_img)

# 키 입력 대기 (0이면 키를 누를 때까지 계속 대기)
cv2.waitKey(0)

# 모든 창 닫기
cv2.destroyAllWindows()

In [5]:
# cv2.imwrite(file_name, image) : 특정한 이미지를 파일로 저장하는 함수
# file_name : 저장할 이미지 파일 이름
# image : 저장할 이미지 객체

cv2.imwrite('test_img.png', img)

True

In [6]:
# cv2.waitKey(time) : 키보드 입력 처리하는 함수
# time : 입력 대기 시간(무한대기 : 0)

In [7]:
# cv2.destroyAllWindows() : 화면의 모든 윈도우를 닫는 함수

In [13]:
import cv2
from ultralytics import YOLO

# YOLO 모델 로드
model = YOLO('yolov9s.pt')

# 웹캠 열기
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('ERROR : Could not open webcam.')
    exit()
    
# 중복없이 탐지된 라벨을 저장할 집합(set)
detected_labels = set()

while True:
    # 프레임 읽기
    ret, frame = cap.read() # ret : 프레임이 성공적으로 읽혔으면 True, 아니면 False
                            # frame : 캡처된 프레임 자체, 이미지로 Numpy 배열로 표현
    if not ret:
        print('ERROR : Could not read frame.')
        break
    
    # 객체 탐지 수행
    results = model.predict(frame, conf=0.5) # conf : 신뢰도가 0.5이상인 경우 탐지된 것으로 간주
    
    # 예측 결과를 프레임에 그리기 및 집합에 라벨 추가
    for result in results:
        for box in result.boxes: # result.boxes : 객체 탐지 모델이 탐지한 각 객체의 경계 상자(좌표)
            x1, y1, x2, y2 = map(int, box.xyxy[0]) # x1, y1 : 왼쪽 상단 좌표, x2, y2 : 오른쪽 하단 좌표, 정수 변환
            conf = box.conf[0]                     # 실제로 해당 클래스에 속할 확률을 나타내는 신뢰도
            label = result.names[int(box.cls[0])]  # 예측된 클래스의 ID
            
            # 바운딩 박스와 레이블 그리기
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) 
            # 사각형(바운딩 박스)그림, 왼쪽상단 좌표, 
            # 오른쪽 하단 좌표, 사각형의 색상, 선의 두께
            cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            # 탐지된 객체의 레이블과 신뢰도 출력, 
            # 객체의 레이블, 신뢰도, 텍스트를 그릴 위치, 사용할 글꼴, 글꼴 크기, 텍스트 색상, 선의 두께
            
            # 탐지된 라벨 집합에 추가(중복 제거)
            detected_labels.add(label)
            
    # 결과 프레임을 화면에 표시
    cv2.imshow('YOLO Test Screen', frame)
        
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 리소스 해제
cap.release() # 웹캠 사용된 리소스 해제
cv2.destroyAllWindows()

# 탐지된 라벨 출력
print('Detected Labels:', detected_labels)


0: 480x640 2 persons, 35.2ms
Speed: 2.5ms preprocess, 35.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 33.6ms
Speed: 1.3ms preprocess, 33.6ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 35.2ms
Speed: 1.0ms preprocess, 35.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 35.2ms
Speed: 1.1ms preprocess, 35.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 34.2ms
Speed: 2.0ms preprocess, 34.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 35.2ms
Speed: 1.5ms preprocess, 35.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 33.9ms
Speed: 1.1ms preprocess, 33.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 33.1ms
Speed: 1.0ms preprocess, 33.1ms inference, 2.0ms postprocess per image at shape (


0: 480x640 2 persons, 36.1ms
Speed: 1.0ms preprocess, 36.1ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 32.2ms
Speed: 1.0ms preprocess, 32.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 35.1ms
Speed: 1.0ms preprocess, 35.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 34.2ms
Speed: 1.5ms preprocess, 34.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 33.1ms
Speed: 1.0ms preprocess, 33.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 34.2ms
Speed: 1.0ms preprocess, 34.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 32.7ms
Speed: 1.0ms preprocess, 32.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 33.6ms
Speed: 0.5ms preprocess, 33.6ms inference, 1.0ms postprocess per image at shape (

In [14]:
# 이미지의 인덱스, 이미지 크기, 탐지된 객체의 수와 종류, 이미지 처리에 걸린시간
# 이미지 전처리, 모델 추론, 결과 후처리에 걸린 시간 / 처리된 이미지의 형태